## Main Authors:
* Fabbrico Davide
* Khateeb Ahmad Arsalan
* Pagnoncelli Diego

In [1]:
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import datetime
import plotly.express as px
import plotly.io as pio

pd.set_option('display.max_rows', 1000)
pd.set_option('mode.chained_assignment', None)
pio.templates.default = "ggplot2"

In [2]:
# read total dataframe
df_tot = pd.read_csv('total_dataframe.csv').drop('Unnamed: 0', axis=1)

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [16]:
df_tot['PRODUCT_CODE'] = df_tot['PRODUCT_CODE'].apply(lambda x: str(x))

## How may country are involve for product x?

In [22]:
products = df_tot.PRODUCT_CODE.unique()
products = pd.Series([p[:2] for p in products])
df_tot['PRODUCT_CODE2'] = df_tot['PRODUCT_CODE'].apply(lambda x: str(x)[:2])

for p in products.unique():
    print(p, len(df_tot[df_tot['PRODUCT_CODE2']==p]))

21 372403
22 511476
39 2890104
42 770247
61 2496288
62 2677145
63 906559
65 210782
73 2466167
84 8485365
85 7015550
90 3356034
95 1183051
03 436719
38 753284
40 1398802
71 412369
83 955844
97 3686471
49 543137
07 249898
08 246628
09 532823
12 644419
15 215745
16 802858
17 296975
18 382526
19 361878
20 496929
25 260295
33 860399
34 835076
35 246066
48 1277194
51 137538
64 714781
69 764183
70 830454
76 580274
82 1389646
87 1678541
94 2322389
96 2357780
27 213816
28 684686
30 615875
36 25552
44 688348
56 764198
59 277843
68 527302
88 117818
91 1071087
99 504262
11 182519
29 1513329
31 111836
32 773284
52 366453
54 509712
74 444700
92 401185
02 153158
13 994225
55 661836
72 759797
89 2153002
93 2525533
14 482003
05 174499
81 565018
57 209515
50 24083
41 122783
43 49001
04 345724
23 116397
37 84376
67 85816
80 41138
45 116535
58 280737
66 61893
53 305706
06 79545
10 73268
01 118825
60 200164
78 17608
75 73645
86 83233
26 33379
46 99793
79 49274
47 44622
24 62599
00 32866
1 4783


In [34]:
df_tot[(df_tot['PRODUCT_CODE2']=='72') & (df_tot['TYPE_FLOW']=='export')]['DECLARANT_ISO'].unique()

array(['FR', 'NL', 'DE', 'IT', 'GB', 'IE', 'DK', 'GR', 'PT', 'ES', 'BE',
       'LU', 'SE', 'FI', 'AT', 'EE', 'LV', 'LT', 'PL', 'CZ', 'SK', 'HU',
       'RO', 'BG', 'SI', 'HR', 'MT', 'CY'], dtype=object)

In [26]:
def PlotGreenGraph(G):
    labels = {e: str(e) for e in G.edges}
    # pos = nx.spring_layout(G, k=10000/np.sqrt(G.order()), iterations=20, scale=10) problem divided by 0
    pos = nx.spring_layout(G, iterations=20, scale=10)
    plt.figure(3, figsize=(20, 14))
    
    nx.draw(G, pos=pos, with_labels=True, node_size=2000, node_color='orange')

    # Create edge labels
    labels = {e: G.edges[e]['Quantity'] for e in G.edges}

    # Draw edge labels according to node positions
    nx.draw_networkx_edge_labels(G, pos, edge_labels=labels)
    
    return None

In [27]:
def GreennessGraph(df, month = 1, year = 2019, type_flow = 'export', product = None, t = 1):
    
    # product
    if product != None or product != '':
        df = df[df['PRODUCT_CODE2']==product].reset_index(drop=True)
    else:
        return 'Select the product code!'
    
    # month and year
    if month == None or year == None or month > 12 or year > 2021 or year < 2019:
        return 'Insert correctly the month and year!'
    
    if month != None and year != None:
        df = df[(df['MONTH'] == month) & (df['YEAR'] == year)].reset_index(drop=True)

    df = df[df['TYPE_FLOW'] == type_flow].reset_index(drop=True) 
    
    df['Distance'] = round(df['Distance'], 3)
    
    # start changes
    df = df.groupby(['DECLARANT_ISO', 'PARTNER_ISO', 'PRODUCT_CODE','TYPE_TRANSP_MODE', 'Distance'])[['DECLARANT_ISO', 'PARTNER_ISO', 'VALUE_IN_EUROS', 'QUANTITY_IN_KG']].sum().reset_index()
    df = df.loc[df['TYPE_TRANSP_MODE'].isin(['Road', 'Air', 'Sea', 'Rail','Inland Waterway'])].reset_index(drop=True)
    speed = 45
    
    def greeness(row):
        if row['TYPE_TRANSP_MODE'] == 'Sea':
            return (row['QUANTITY_IN_KG']*(row['Distance']/1000)*9936.70)/((24000*7000)*speed)
        elif row['TYPE_TRANSP_MODE'] == 'Air':
            return (row['QUANTITY_IN_KG']*12*(row['Distance']/1000))/77000
        # elif row['TYPE_TRANSP_MODE'] == 'Inland Waterway':
            # return (row['QUANTITY_IN_KG']*0.12*(row['Distance']/1000)) # find the maximum capacity
        elif row['TYPE_TRANSP_MODE'] == 'Road':
            return (row['QUANTITY_IN_KG']*(row['Distance']/1000)*0.25)/(30000)
        elif row['TYPE_TRANSP_MODE'] == 'Rail':
            return (row['QUANTITY_IN_KG']*(row['Distance']/1000)*7.97)/(528000)

    df['Greenness'] = df.apply(lambda row: greeness(row), axis=1) 
    
    tr = int(len(df)*t/100)
    df = df.iloc[:tr+1]
    
    G = nx.DiGraph()
    # nodes
    nodes = df['DECLARANT_ISO'].append(df['PARTNER_ISO']).unique()
    
    # graph
    G.add_nodes_from(nodes)

    for index,row in df.iterrows():
        G.add_edge(row['DECLARANT_ISO'],row['PARTNER_ISO'],
                                             Value = row.VALUE_IN_EUROS,
                                             Quantity = row.QUANTITY_IN_KG,
                                            Distance = row.Distance,
                                            Greenness = row.Greenness)

    
    return G

In [28]:
def DFGreenGraph(G, month=None, year=None):
    dict_summ = {}
    for i in G.nodes():
        summ = 0
        for j in G[i]:
            summ += G[i][j]['Greenness'] # for each node I calculate the sum of the GREENNESS of the out-edge
        dict_summ[i] = summ
        
    dict_summQ = {}
    for i in G.nodes():
        summQ = 0
        for j in G[i]:
            summQ += G[i][j]['Quantity'] # for each node I calculate the sum of the GREENNESS of the out-edge
        dict_summQ[i] = summQ

    df_greenG = pd.DataFrame(list(dict_summ.items()), columns=['Country', 'Greenness']).sort_values(by='Greenness', ascending=False).reset_index(drop=True)
    df_greenQ = pd.DataFrame(list(dict_summQ.items()), columns=['Country', 'Quantity'])
    df_GQ = df_greenG.merge(df_greenQ, on='Country')
    df_GQ['month'] = month
    df_GQ['year'] = year
    df_GQ['date'] = datetime.datetime(year, month, 1)
    df_GQ = df_GQ[df_GQ['Greenness']>0]
    return df_GQ

In [35]:
year = [2019, 2020, 2021]
month = [1,2,3,4,5,6,7,8,9,10,11,12]
df = pd.DataFrame({})
for y in year:
    print(str(y))
    if y == 2021:
        month = list(range(1, 7))   
    for m in month:
        print(str(m))
        greenG = GreennessGraph(df_tot, m, y, product = '97', t = 40)
        greenDF = DFGreenGraph(greenG, month=m, year=y)
        df = pd.concat([df, greenDF], copy=False)
df = df.reset_index(drop=True)

2019
1
2
3
4
5
6
7
8
9
10
11
12
2020
1
2
3
4
5
6
7
8
9
10
11
12
2021
1
2
3
4
5
6


In [84]:
df1 = df.copy()

In [65]:
fr = df1[df1['Country']=='FR']
fr

,Country,Greenness,Quantity,month,year,date
50,FR,2514.301243,204822,5,2019,2019-05-01
71,FR,327.833101,69570,7,2019,2019-07-01
82,FR,22.453816,6905,8,2019,2019-08-01
93,FR,3001.259811,72805,9,2019,2019-09-01
104,FR,0.139828,43,10,2019,2019-10-01
125,FR,1546.554021,225096,12,2019,2019-12-01


In [85]:
df_row = pd.DataFrame({})
year = [2019, 2020, 2021]
month = [1,2,3,4,5,6,7,8,9,10,11,12]
for y in year:
    if y == 2021:
        month = list(range(1, 7))   
    for m in range(len(month)):
        df_row = df_row.append({'Country':'FR', 'Greenness':0, 'Quantity':0, 'month':month[m], 'year':y}, ignore_index=True)
df_row

,Country,Greenness,Quantity,month,year
0,FR,0.0,0.0,1.0,2019.0
1,FR,0.0,0.0,2.0,2019.0
2,FR,0.0,0.0,3.0,2019.0
3,FR,0.0,0.0,4.0,2019.0
4,FR,0.0,0.0,5.0,2019.0
5,FR,0.0,0.0,6.0,2019.0
6,FR,0.0,0.0,7.0,2019.0
7,FR,0.0,0.0,8.0,2019.0
8,FR,0.0,0.0,9.0,2019.0
9,FR,0.0,0.0,10.0,2019.0


In [86]:
df1 = df1[(df1['Country']!='FR') & (df1['Country']!='AT')]

In [87]:
df_country = pd.DataFrame({})
for i in df1['Country'].unique():
    df_country[i] = df1[df1['Country'] == i]['Greenness'].to_list()
    
for i in df1['Country'].unique():
    df_country['year'] = df1[df1['Country'] == i]['year'].to_list()
    df_country['month'] = df1[df1['Country'] == i]['month'].to_list()

## Multiple Line

In [88]:
hmc = len(df1['Country'].unique())
fig = px.line(df_country, x="month", y=df_country.columns[:hmc], animation_frame="year", markers=True)
fig["layout"]
fig.layout.updatemenus[0].buttons[0].args[1]["frame"]["duration"] = 1500
fig.update_layout(
    title="Index of Greenness (2019-2021)",
    xaxis_title="Month",
    yaxis_title="Greenness",
    title_x=0.5,
    font=dict(
        size=16,
    ),
    yaxis_range=[min(df1['Greenness']-3000),max(df1['Greenness']+50000)],
    xaxis_range = [0.5,12.5]
)
# fig.update_layout(title_text='Index of Greenness (2019-2021)', title_x=0.5)
fig.update_xaxes(tickmode = 'array',
                 tickvals = sorted(df1['month'].unique()),
                 ticktext= ['Jan', 'Feb', 'Mar', 'Apr', 'May','Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])
fig.write_html("/Users/davide/Desktop/moreline.html")

## One Line

In [89]:
df2 = df1[df1['Country']=='BE']
fig = px.line(df2, x="month", y="Greenness", animation_frame="year", animation_group="Country", hover_name="Country", markers=True)
fig["layout"]
fig.layout.updatemenus[0].buttons[0].args[1]["frame"]["duration"] = 1500
fig.update_layout(
    title="Index of Greenness (2019-2021)",
    xaxis_title="Month",
    yaxis_title="Greenness",
    title_x=0.5,
    font=dict(
        size=16,
    ),
    yaxis_range=[min(df2['Greenness']-3000),max(df2['Greenness']+111000)],
    xaxis_range = [0.5,12.5]
)
# fig.update_layout(title_text='Index of Greenness (2019-2021)', title_x=0.5)
fig.update_xaxes(tickmode = 'array',
                 tickvals = sorted(df2['month'].unique()),
                 ticktext= ['Jan', 'Feb', 'Mar', 'Apr', 'May','Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])
fig.write_html("/Users/davide/Desktop/oneline.html")

In [90]:
ccode_df = pd.read_csv('country_codes.csv')
map_dict1 = pd.Series(ccode_df.iloc[:,0].values,index= ccode_df.iloc[:,1]).to_dict()
map_dict2 = pd.Series(ccode_df.iloc[:,2].values,index= ccode_df.iloc[:,0]).to_dict()
df1.replace({"Country":map_dict1},inplace=True)
df1['Code'] = df1['Country'].map(map_dict2)
df1['Date'] = df1['date'].dt.strftime("%b-%Y").astype(str)

In [91]:
fig = px.choropleth(df1, locations="Code",
                    scope = "europe",
                    color="Greenness", # lifeExp is a column of gapminder
                    hover_name="Country", # column to add to hover information
                    color_continuous_scale="Greens",
                    animation_frame="Date"
                    )
fig.update_layout(margin={"r":20,"t":20,"l":2,"b":200})
fig.show()
fig.write_html("/Users/davide/Desktop/map.html")
#fig.write_html("C:\\Users\\Arsalan Khateeb\\Coding_Lab\\sta-py\\file.html")